## EDF file to stages conversion 

In [1]:
import sys

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import os
import numpy as np
import mne

In [4]:
# CHANGE HERE
SUBJ_NUM = 44

# Type of the test (math / video)
TEST_TYPE = 'math' 


In [5]:
# Initalize path variables for main folders

print(os.getcwd())

if (SUBJ_NUM < 10):
    subj_dir_name = '0'+str(SUBJ_NUM)+'_'+TEST_TYPE
else:
    subj_dir_name = str(SUBJ_NUM)+'_'+TEST_TYPE
    
root_dir_path = os.path.join(os.getcwd(), 'work_data', subj_dir_name)
init_dir_path = os.path.join(root_dir_path, 'initial_data')
raw_dir_path = os.path.join(root_dir_path, 'stages_raw')
ep_dir_path = os.path.join(root_dir_path, 'stages_epochs')
ft_dir_path = os.path.join(root_dir_path, 'features')

print(init_dir_path)
print(raw_dir_path)
print(ep_dir_path)
print(ft_dir_path)


/home/user/common/dsukhinin/eeg3/Huawei
/home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/initial_data
/home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/stages_raw
/home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/stages_epochs
/home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/features


In [6]:
# Create main folders if not exist

if not os.path.exists(root_dir_path):
    os.mkdir(root_dir_path)
if not os.path.exists(init_dir_path):
    os.mkdir(init_dir_path)
if not os.path.exists(raw_dir_path):
    os.mkdir(raw_dir_path)
if not os.path.exists(ep_dir_path):
    os.mkdir(ep_dir_path)
if not os.path.exists(ft_dir_path):
    os.mkdir(ft_dir_path)


In [7]:
# Frequency bands

bands = [(0.9, 4, 'Delta (0.9-4 Hz)', 'D'), (4, 8, 'Theta (4-8 Hz)', 'T'), (8, 14, 'Alpha (8-14 Hz)', 'A'), 
         (14, 25, 'Beta (14-25 Hz)', 'B'), (25, 40, 'Gamma (25-40 Hz)', 'G')]

str_freq = [bands[i][3] for i in range(len(bands))]

In [8]:
# Localization by scalp regions

regions = [(['Fp1','Fp2'], 'Fp', 'Pre-frontal'), (['F7','F3','FC5'], 'LF', 'Left Frontal'), 
           (['Fz','FC1','FC2'], 'MF', 'Midline Frontal'), (['F4','F8','FC6'], 'RF', 'Right Frontal'),
           (['T7','CP5','P7'], 'LT', 'Left Temporal'), (['T8','CP6','P8'], 'RT', 'Right Temporal'), 
           (['C3','Cz','C4'], 'Cen', 'Central'), (['P3','Pz','P4','CP1','CP2'], 'Par', 'Parietal'), 
           (['O1','Oz','O2'], 'Occ', 'Occipital')]

SLICE_LEN = 10 #number of epochs to measure physiological features, coherence and PLV

n_freq = len(str_freq)
n_regions = len(regions)


# Loading data

In [9]:
# Load and analise original EEG and biological data

EOG_ch = ['HEOG', 'VEOG']
ECG_ch = ['heartrate']
resp_ch = ['respiratory']
marker_ch = ['marker']
zygomat_ch = ['zygomaticus', 'corrugator']
skincond_ch = ['skinconductance']
corrug_ch = ['corrugator']

# Compose initial data edf-filename
edf_fname = str(SUBJ_NUM)+'_'+TEST_TYPE+'-edf.edf'
#edf_fname = str(SUBJ_NUM)+'_'+TEST_TYPE+'.vhdr'
edf_file_path = os.path.join(init_dir_path, edf_fname)
print(edf_file_path)

# Load data
data_raw = mne.io.read_raw_edf(os.path.join(os.getcwd(), 'work_data', edf_fname), preload=True)

samp_rate = data_raw.info['sfreq']


# Set channel types
change_types = {'HEOG':'eog', 'VEOG':'eog', 'heartrate':'bio', 'respiratory':'bio', 'zygomaticus':'bio', 'corrugator':'bio',
                'skinconductance':'bio', 'Photosensor':'bio', 'Sync':'bio'}
data_raw.set_channel_types(change_types)

#data_raw.drop_channels(['marker'])
data_raw.rename_channels({'FP2':'Fp2'})

data_raw.set_montage('standard_1020')


/home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/initial_data/44_math-edf.edf
Extracting EDF parameters from /home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math-edf.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2342499  =      0.000 ...  4684.998 secs...


<RawEDF | 44_math-edf.edf, 37 x 2342500 (4685.0 s), ~661.3 MB, data loaded>

In [10]:
annot = data_raw.annotations.copy()
data_raw.crop(tmin=annot.onset[1], tmax=annot.onset[-1]+1)

<RawEDF | 44_math-edf.edf, 37 x 2095898 (4191.8 s), ~591.7 MB, data loaded>

In [11]:
ann = data_raw.annotations.copy()
print(len(ann))

del_ann_ind = []
for i in range(len(ann)):
    if (ann[i]['description'] in [ 'New Segment', 'S  8', 'S 66', 'S 69', 'S 63', 'S 77', 'S 79', 'S 73']):
        del_ann_ind += [i]
data_raw.annotations.delete(del_ann_ind)

321


In [12]:
# Cropping data (event-related)
annot = data_raw.annotations.copy()
annot_onsets = annot.onset - data_raw.first_time

baseline_main_raw = data_raw.copy().crop(tmin=annot_onsets[0], tmax=annot_onsets[1])

# Stages & stage baselines
stages_raw = []
stage_types = []
for i in range(len(annot)):
    
    if (annot.description[i] in {'S  5', 'S  6', 'S  7'}):

        stages_raw.append(data_raw.copy().crop(tmin=annot_onsets[i], tmax=annot_onsets[i+1]))
        stage_types.append(int(annot.description[i][-1]))
        print(annot.description[i], annot.description[i+1], 
              stages_raw[-1].n_times/samp_rate)
         
# Initialize n_stages
n_stages = len(stage_types)

S  7 S 70 223.09
S  6 S 60 180.686
S  5 S 50 180.002
S  6 S 60 191.952
S  7 S 70 189.086
S  5 S 50 180.002
S  6 S 60 192.856
S  7 S 70 180.602
S  5 S 50 180.002
S  6 S 60 184.218
S  7 S 70 203.254
S  5 S 50 180.004
S  7 S 70 182.604
S  5 S 50 180.002
S  6 S 60 195.836
S  7 S 70 182.12
S  6 S 60 194.872
S  5 S 50 180.004


In [13]:
# Saving filtered and cropped raw data

# Initialize n_stages
n_stages = len(stage_types)

raw_dir_path = os.path.join(root_dir_path, 'stages_raw')
if not os.path.exists(raw_dir_path):
    os.mkdir(raw_dir_path)

# Stage types
np.savetxt(os.path.join(raw_dir_path, 'stage_types.txt'), stage_types)

# Raw data
for _st in range(n_stages):
    stages_raw[_st].save(os.path.join(raw_dir_path, 'st_'+str(_st+1)+'_raw.fif'), overwrite=True)
    print(stages_raw[_st].get_data().shape)

baseline_main_raw.save(os.path.join(raw_dir_path, 'bl_main_raw.fif'), overwrite=True)


Overwriting existing file.
Writing /home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/stages_raw/st_1_raw.fif
Closing /home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/stages_raw/st_1_raw.fif
[done]
(37, 111545)
Overwriting existing file.
Writing /home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/stages_raw/st_2_raw.fif
Closing /home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/stages_raw/st_2_raw.fif
[done]
(37, 90343)
Overwriting existing file.
Writing /home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/stages_raw/st_3_raw.fif
Closing /home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/stages_raw/st_3_raw.fif
[done]
(37, 90001)
Overwriting existing file.
Writing /home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/stages_raw/st_4_raw.fif
Closing /home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/stages_raw/st_4_raw.fif
[done]
(37, 95976)
Overwriting existing file.
Writing /home/user/common/dsukhinin/eeg3/Huawei/work_data/44_math/st

In [14]:
print("processed", subj_dir_name)
exit()

processed 44_math
